In [ ]:
import os

os.getcwd()
os.chdir("../../")
os.getcwd()

In [ ]:
import sys
import argparse
import os

sys.argv = ["view", "--config", "config/single_task_object_detection.yaml"]

parser = argparse.ArgumentParser()
parser.add_argument("--config", type=str, required=True, help="Path to the config file")
args = parser.parse_args()

print(args.config)

In [ ]:
import torch
import torch.nn.functional as F
import torch.nn as nn

In [ ]:
W = torch.arange(12, dtype=torch.float32).reshape(4, 3)
b = torch.arange(4, dtype=torch.float32)
print(W)
print(b)

# SVD


In [ ]:
U, S, V = torch.svd(W)

In [ ]:
print(U)
print(S)
print(torch.diag(S))
print(V)

In [ ]:
U @ torch.diag(S) @ V.t()  # deve essere un'approssimazione di W

# Truncated SVD


In [ ]:
U, S, V = torch.svd(W)

In [ ]:
t = 2
St = S[:t]
St_diag = torch.diag(St)

In [ ]:
print(St)
print(St_diag)

In [ ]:
# pad: left, right, up, down
# aggiunge un numero di colonne e righe di padding specificato  in questo caso solo una a destra e una sotto
St_diag = F.pad(input=St_diag, pad=(0, 1, 0, 1), mode="constant", value=0)
print(St_diag)

In [ ]:
U @ St_diag @ V.t()  # deve essere un'approssimazione di W

To compress a network, the single fully connected layer corresponding to W is replaced by two fully connected layers, without a non-linearity between them. The first of these layers uses the weight matrix ΣtV T (and no biases) and the second uses U (with the original bi- ases associated with W ). This simple compression method gives good speedups when the number of RoIs is large.


# change weights matrix to FC layers


In [ ]:
W = torch.arange(12, dtype=torch.float32).reshape(4, 3)
b = torch.arange(4, dtype=torch.float32)
fc = torch.nn.Linear(3, 4, bias=True)
fc.weight = torch.nn.Parameter(W)
fc.bias = torch.nn.Parameter(b)
print(fc.weight)
print(fc.bias)

In [ ]:
U, S, V = torch.svd(fc.weight)
t = 2
St = S[:t]
St_diag = torch.diag(St)
St_diag = F.pad(input=St_diag, pad=(0, 1, 0, 1), mode="constant", value=0)

In [ ]:
desired_weights = St_diag @ V.t()
input_size = desired_weights.size(1)
output_size = desired_weights.size(0)
first_fc_layer = nn.Linear(input_size, output_size, bias=False)

with torch.no_grad():
    first_fc_layer.weight = nn.Parameter(desired_weights)

print("Pesi:", first_fc_layer.weight)
print("Bias:", first_fc_layer.bias)

In [ ]:
desired_weights = U
desired_bias = fc.bias

input_size = desired_weights.size(1)
output_size = desired_weights.size(0)
fc_layer = nn.Linear(input_size, output_size)

with torch.no_grad():
    fc_layer.weight = nn.Parameter(desired_weights)
    fc_layer.bias = nn.Parameter(desired_bias)

# Stampa pesi e bias per verificare
print("Pesi:", fc_layer.weight)
print("Bias:", fc_layer.bias)

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F


def truncated_svd_decomposition(layer, t):
    """
    Applies truncated SVD decomposition on the given linear layer.

    Args:
    - layer (nn.Linear): The linear layer to be decomposed.
    - t (int): Number of singular values to keep.

    Returns:
    - nn.Sequential: A sequential model with two linear layers representing the truncated SVD.
    """
    # Perform SVD on the weight matrix
    W = layer.weight.data
    U, S, V = torch.svd(W)

    # Keep only the top t singular values
    U_t = U[:, :t]
    S_t = S[:t]
    V_t = V[:, :t]

    # Create the two new linear layers
    first_layer = nn.Linear(V_t.shape[0], t, bias=False)
    second_layer = nn.Linear(t, U_t.shape[0], bias=True)

    # Initialize the weights of the new layers
    first_layer.weight.data = (S_t.unsqueeze(1) * V_t.t()).t()
    second_layer.weight.data = U_t.t()

    # Set the bias of the second layer to be the same as the original layer
    second_layer.bias.data = layer.bias.data.clone()

    # Return a sequential model of the two layers
    return nn.Sequential(first_layer, second_layer)


# Example usage
original_layer = nn.Linear(4096, 4096)
compressed_layer = truncated_svd_decomposition(original_layer, t=256)

In [ ]:
original_layer.weight[:20, :20]

In [ ]:
from model import ObjectDetectionModel as ObjModelFull
from model_compressed import ObjectDetectionModel as ObjModelCompressed

In [ ]:
def count_parameters(model):
    """
    Conta il numero di parametri nel modello.

    Args:
    - model (nn.Module): Il modello PyTorch di cui contare i parametri.

    Returns:
    - int: Il numero totale di parametri nel modello.
    """
    return sum(p.numel() for p in model.parameters())

In [ ]:
count_parameters(ObjModelFull())

In [ ]:
count_parameters(ObjModelCompressed())